In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train = pd.read_excel('Data_Train.xlsx')
test = pd.read_excel('Test_set.xlsx')

In [3]:
train.shape, test.shape

((10683, 11), (2671, 10))

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [5]:
train.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [6]:
for i in train.columns:
    print("Unique values in", i, train[i].nunique())

Unique values in Airline 12
Unique values in Date_of_Journey 44
Unique values in Source 5
Unique values in Destination 6
Unique values in Route 128
Unique values in Dep_Time 222
Unique values in Arrival_Time 1343
Unique values in Duration 368
Unique values in Total_Stops 5
Unique values in Additional_Info 10
Unique values in Price 1870


In [7]:
for i in test.columns:
    print("Unique values in", i, test[i].nunique())

Unique values in Airline 11
Unique values in Date_of_Journey 44
Unique values in Source 5
Unique values in Destination 6
Unique values in Route 100
Unique values in Dep_Time 199
Unique values in Arrival_Time 704
Unique values in Duration 320
Unique values in Total_Stops 5
Unique values in Additional_Info 6


## Data Pre Processing

In [8]:
train_df = train[['Airline', 'Source', 'Destination', 'Total_Stops', 'Additional_Info', 'Date_of_Journey', 'Dep_Time', 
                  'Route', 'Arrival_Time', 'Price']]
test_df = test[['Airline', 'Source', 'Destination', 'Total_Stops', 'Additional_Info', 'Date_of_Journey', 'Dep_Time', 
                'Route', 'Arrival_Time']]

In [9]:
Class = {'IndiGo': 'Economy',
         'GoAir': 'Economy',
         'Vistara': 'Economy',
         'Vistara Premium economy': 'Premium Economy',
         'Air Asia': 'Economy',
         'Trujet': 'Economy',
         'Jet Airways': 'Economy',
         'SpiceJet': 'Economy',
         'Jet Airways Business': 'Business',
         'Air India': 'Economy',
         'Multiple carriers': 'Economy',
         'Multiple carriers Premium economy': 'Premium Economy'}
train_df['Booking_Class'] = train_df['Airline'].map(Class)
test_df['Booking_Class'] = test_df['Airline'].map(Class)

In [10]:
market = {'IndiGo': 41.3,
         'GoAir': 8.4,
         'Vistara': 3.3,
         'Vistara Premium economy': 3.3,
         'Air Asia': 3.3,
         'Trujet': 0.1,
         'Jet Airways': 17.8,
         'SpiceJet': 13.3,
         'Jet Airways Business': 17.8,
         'Air India': 13.5,
         'Multiple carriers': 1,
         'Multiple carriers Premium economy': 1}
train_df['Market_Share'] = train_df['Airline'].map(market)
test_df['Market_Share'] = test_df['Airline'].map(market)

In [11]:
df1 = train_df.copy() 
df1['Day_of_Booking'] = '1/3/2019'
df1['Day_of_Booking'] = pd.to_datetime(df1['Day_of_Booking'],format='%d/%m/%Y')
df1['Date_of_Journey'] = pd.to_datetime(df1['Date_of_Journey'],format='%d/%m/%Y')
df1['Days_to_Departure'] = (df1['Date_of_Journey'] - df1['Day_of_Booking']).dt.days
train_df['Days_to_Departure'] = df1['Days_to_Departure']

df2 = test_df.copy() 
df2['Day_of_Booking'] = '1/3/2019'
df2['Day_of_Booking'] = pd.to_datetime(df2['Day_of_Booking'],format='%d/%m/%Y')
df2['Date_of_Journey'] = pd.to_datetime(df2['Date_of_Journey'],format='%d/%m/%Y')
df2['Days_to_Departure'] = (df2['Date_of_Journey'] - df2['Day_of_Booking']).dt.days
test_df['Days_to_Departure'] = df2['Days_to_Departure']

del df1, df2

In [12]:
# Let's take only Arrial Time (withut including date)
train_df['Arrival_Time'] = train['Arrival_Time'].str.split(' ').str[0]
test_df['Arrival_Time'] = test['Arrival_Time'].str.split(' ').str[0]

In [13]:
def get_departure(dep):
    dep = dep.split(':')
    dep = int(dep[0])
    if (dep >= 6 and dep < 12):
        return 'Morning'
    elif (dep >= 12 and dep < 17):
        return 'Noon'
    elif (dep >= 17 and dep < 20):
        return 'Evening'
    else:
        return 'Night'
    
train_df['Dep_timeofday'] = train['Dep_Time'].apply(get_departure)   
test_df['Dep_timeofday'] = test['Dep_Time'].apply(get_departure) 

train_df['Arr_timeofday'] = train['Arrival_Time'].apply(get_departure)   
test_df['Arr_timeofday'] = test['Arrival_Time'].apply(get_departure) 

In [14]:
train_df['Total_Stops'] = train_df['Total_Stops'].str.replace('non-stop','0')
train_df['Total_Stops'] = train_df['Total_Stops'].str.replace('stops','')
train_df['Total_Stops'] = train_df['Total_Stops'].str.replace('stop','')
train_df['Total_Stops'].fillna(0, inplace=True)   
train_df['Total_Stops'] = train_df['Total_Stops'].astype(float)

test_df['Total_Stops'] = test_df['Total_Stops'].str.replace('non-stop','0')
test_df['Total_Stops'] = test_df['Total_Stops'].str.replace('stops','')
test_df['Total_Stops'] = test_df['Total_Stops'].str.replace('stop','')
#test_df['Total_Stops'].fillna(0, inplace=True)
test_df['Total_Stops'] = test_df['Total_Stops'].astype(float)

In [24]:
train_df['Hours'] = train['Duration'].str.split(' ').str[0]
train_df['Hours'] = train_df['Hours'].str.replace('h','').astype(float)
train_df['Hours'].fillna(0, inplace=True) 

train_df['Minutes'] = train['Duration'].str.split(' ').str[1]
train_df['Minutes'] = train_df['Minutes'].str.replace('m','').astype(float)
train_df['Minutes'].fillna(0, inplace=True)

test_df['Hours'] = test['Duration'].str.split(' ').str[0]
test_df['Hours'] = test_df['Hours'].str.replace('h','').astype(float)
test_df['Hours'].fillna(0, inplace=True) 

test_df['Minutes'] = test['Duration'].str.split(' ').str[1]
test_df['Minutes'] = test_df['Minutes'].str.replace('m','').astype(float)
test_df['Minutes'].fillna(0, inplace=True)

train_df['Hours'] = train_df['Hours'] * 60
train_df['Duration'] = train_df['Hours'] + train_df['Minutes']

test_df['Hours'] = test_df['Hours'] * 60
test_df['Duration'] = test_df['Hours'] + test_df['Minutes']

train_df.drop(['Hours', 'Minutes'], axis=1, inplace=True)
test_df.drop(['Hours', 'Minutes'], axis=1, inplace=True)

In [29]:
train_df['Price'] = np.log1p(train_df['Price'])

train_df['Duration'] = np.log1p(train_df['Duration'])
test_df['Duration'] = np.log1p(test_df['Duration'])

10683

In [30]:
train_df = pd.get_dummies(train_df, columns=['Airline', 'Source', 'Destination', 'Additional_Info', 'Date_of_Journey',
                                             'Dep_Time', 'Arrival_Time', 'Dep_timeofday', 'Booking_Class', 'Arr_timeofday'],
                          drop_first=True)
test_df = pd.get_dummies(test_df, columns=['Airline', 'Source', 'Destination', 'Additional_Info', 'Date_of_Journey',
                                           'Dep_Time', 'Arrival_Time', 'Dep_timeofday', 'Booking_Class', 'Arr_timeofday'],
                         drop_first=True)

In [31]:
def clean_route(route):
    route = str(route)
    route = route.split(' → ')
    return ' '.join(route)

train_df['Route'] = train_df['Route'].apply(clean_route)
test_df['Route'] = test_df['Route'].apply(clean_route)

from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(ngram_range=(1, 1), lowercase=False)
train_route = tf.fit_transform(train_df['Route'])
test_route = tf.transform(test_df['Route'])

train_route = pd.DataFrame(data=train_route.toarray(), columns=tf.get_feature_names())
test_route = pd.DataFrame(data=test_route.toarray(), columns=tf.get_feature_names())

/opt/anaconda3/envs/CSE801/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/opt/anaconda3/envs/CSE801/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [32]:
train_df = pd.concat([train_df, train_route], axis=1) 
train_df.drop('Route', axis=1, inplace=True)

test_df = pd.concat([test_df, test_route], axis=1) 
test_df.drop('Route', axis=1, inplace=True)

In [38]:
import re

train_df = train_df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
test_df = test_df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

train_df.head()


,Total_Stops,Price,Market_Share,Days_to_Departure,Duration,Airline_AirIndia,Airline_GoAir,Airline_IndiGo,Airline_JetAirways,Airline_JetAirwaysBusiness,...,PAT,PNQ,RPR,STV,TRV,UDR,VGA,VNS,VTZ,nan
0,0.0,8.268219,41.3,23,5.141664,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,8.944159,13.5,61,6.100319,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,9.538420,17.8,100,7.039660,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,8.735364,41.3,72,5.786897,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,9.495745,41.3,0,5.655992,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
X = train_df.drop(labels=['Price'], axis=1)
y = train_df['Price'].values

from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.25, random_state=1)

In [40]:
X_train.shape, y_train.shape, X_cv.shape, y_cv.shape

((8012, 570), (8012,), (2671, 570), (2671,))

In [41]:

from math import sqrt 
from sklearn.metrics import mean_squared_log_error

In [42]:
import lightgbm as lgb
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_cv, label=y_cv)

param = {'objective': 'regression',
         'boosting': 'gbdt',
         'num_iterations': 3000,   
         'learning_rate': 0.06,  
         'num_leaves': 40,  
         'max_depth': 24,   
         'min_data_in_leaf':11,  
         'max_bin': 4, 
         'metric': 'l2_root'
         }

lgbm = lgb.train(params=param,
                 verbose_eval=1000,
                 train_set=train_data,
                 valid_sets=[test_data])

y_pred2 = lgbm.predict(X_cv)
print('RMSLE:', sqrt(mean_squared_log_error(np.exp(y_cv), np.exp(y_pred2))))

/opt/anaconda3/envs/CSE801/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/opt/anaconda3/envs/CSE801/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000650 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 865
[LightGBM] [Info] Number of data points in the train set: 8012, number of used features: 407
[LightGBM] [Info] Start training from score 8.988209
[1000]	valid_0's rmse: 0.107026
[2000]	valid_0's rmse: 0.10623
[3000]	valid_0's rmse: 0.106429
RMSLE: 0.10641493529415487


In [55]:
train_df

,Total_Stops,Price,Market_Share,Days_to_Departure,Duration,Airline_AirIndia,Airline_GoAir,Airline_IndiGo,Airline_JetAirways,Airline_JetAirwaysBusiness,...,PAT,PNQ,RPR,STV,TRV,UDR,VGA,VNS,VTZ,nan
0,0.0,8.268219,41.3,23,5.141664,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,8.944159,13.5,61,6.100319,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,9.538420,17.8,100,7.039660,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,8.735364,41.3,72,5.786897,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,9.495745,41.3,0,5.655992,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10678,0.0,8.320692,3.3,39,5.017280,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10679,0.0,8.329899,13.5,57,5.049856,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10680,0.0,8.885994,17.8,57,5.198497,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10681,0.0,9.445333,3.3,0,5.081404,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
